In [1]:
%load_ext watermark
%watermark

2019-04-07T18:05:28-05:00

CPython 3.7.3rc1
IPython 7.3.0

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [3]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [12, 8]

Para garantizar la reproducibilidad (esto es, que al ejecutar este notebook otra vez los resultados sean idénticos) vamos a fijar la semilla que usa scikit learn para hacer particiones aleatorias. Para esto tenemos que fijar la semilla (seed) cada vez que llamemos a la aplicacion.

In [4]:
 np.random.seed(42)

# Selección de variables

En esta sección vamos a ver las distintas técnicas de selección de variables, que son las técnicas que nos permiten filtrar las variables que tenemos y elegir aquellas que realmente están ayudando al modelo.

¿Por qué querriamos eliminar variables? Generalmente, cuanta más información tenemos sobre cada observación mejor será la calidad de nuestras predicciones (cuanto más sepamos, más facil será diferenciar unos casos de otros). No obstante, esto no tiene por que ser así en todas las situaciones, hay veces que ciertas variables pueden ser contraproducentes y hacer que nuestros modelos funcionen peor, por ejemplo debido a errores de medición.

Seleccionar variables tiene varias ventajas. En primer lugar, como hemos dicho, puede mejorar las puntuaciones de los modelos (eliminando parte del ruido en los datos), por otra parte, reducir el número de variables independientes reduce la complejidad del modelo, reduciendo así el efecto de sobreajuste del modelo. Además, tener menos variables independientes y modelos más simples significa que nuestros modelos son más rápidos de entrenar, y podemos entrenarlos con más datos en las mismas máquinas.

## Cargamos los datos

Para esta sección vamos a usar el [Ames Housing dataset](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) que es una versión más moderna y con muchas más variables que el Boston Housing Dataset. Tiene información sobre casas en la ciudad de Ames, en Iowa, y su precio de venta. Este [link](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) contiene la descripción de las variables en inglés.

In [6]:
ames = pd.read_csv("D:/datasets/Curso_Mauel_Garrido/ames.csv").drop("id_parcela", axis=1) # no necesitamos el identificador de la parcela

In [7]:
ames.shape

(2930, 80)

In [8]:
ames.head()

,tipo_construccion,tipo_zona,perimetro_conectado_calle,area_parcela,tipo_acceso,tipo_acceso_trasero,forma_parcela,LandContour,tipo_instalaciones,config_parcela,...,area_piscina,calidad_piscina,calidad_valla,atributo_miscelaneo,valor_atributo_miscelaneo,mes_venta,ano_venta,tipo_venta,condicion_venta,precio_venta
0,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [9]:
ames.shape

(2930, 80)

In [10]:
ames.sample(1).T

,1357
tipo_construccion,70
tipo_zona,RM
perimetro_conectado_calle,NaN
area_parcela,5100
tipo_acceso,Pave
tipo_acceso_trasero,Grvl
forma_parcela,Reg
LandContour,Lvl
tipo_instalaciones,AllPub
config_parcela,Inside


In [11]:
ames.precio_venta.describe()

count      2930.000000
mean     180796.060068
std       79886.692357
min       12789.000000
25%      129500.000000
50%      160000.000000
75%      213500.000000
max      755000.000000
Name: precio_venta, dtype: float64

In [12]:
ames.dtypes

tipo_construccion                int64
tipo_zona                       object
perimetro_conectado_calle      float64
area_parcela                     int64
tipo_acceso                     object
tipo_acceso_trasero             object
forma_parcela                   object
LandContour                     object
tipo_instalaciones              object
config_parcela                  object
pendiente_parcela               object
barrio                          object
cercania_carretera1             object
cercania_carretera2             object
tipo_edificio                   object
tipo_casa                       object
calidad_general                  int64
condicion_general                int64
fecha_construccion               int64
ano_remodelado                   int64
tipo_tejado                     object
material_tejado                 object
material_exterior1              object
material_exterior2              object
tipo_revestimiento              object
area_revestimiento       

## Procesado de datos

Para poder utilizar este dataset tenemos que transformar las variables categóricas en numéricas. En vez de crear nuestro propio `ColumnSelector` vamos a usar el de la libreria auxiliar `mlxtend`

In [20]:
variables_independientes = ames.drop("precio_venta", axis=1).columns
variable_objetivo = ["precio_venta"]

In [27]:
datos_numericos = ames[variables_independientes].select_dtypes(np.number)
col_no_numericas = ames[variables_independientes].select_dtypes([object]).columns

col_numericas = datos_numericos.columns

# para crear este diccionario he ido a la descripcion del dataset
dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
     "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
}
col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = ames[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = ames[col_categoricas]


In [28]:
datos_numericos.head()

,tipo_construccion,perimetro_conectado_calle,area_parcela,calidad_general,condicion_general,fecha_construccion,ano_remodelado,area_revestimiento,area_sotano_habitable1,area_sotano_habitable2,...,area_garage,area_terraza_madera,area_porche_abierto,area_porche_cerrado,area_porche_3estaciones,area_porche_tapado,area_piscina,valor_atributo_miscelaneo,mes_venta,ano_venta
0,20,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,...,528.0,210,62,0,0,0,0,0,5,2010
1,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,730.0,140,0,0,0,120,0,0,6,2010
2,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,312.0,393,36,0,0,0,0,12500,6,2010
3,20,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,...,522.0,0,0,0,0,0,0,0,4,2010
4,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,482.0,212,34,0,0,0,0,0,3,2010


In [29]:
datos_categoricos.head()

,tipo_garaje,sistema_electrico,LandContour,material_exterior2,tipo_tejado,condicion_venta,cercania_carretera1,tipo_casa,material_exterior1,tipo_calefaccion,...,barrio,tipo_acceso_trasero,tipo_zona,atributo_miscelaneo,tipo_andamios,config_parcela,material_tejado,tipo_acceso,tipo_edificio,tipo_revestimiento
0,Attchd,SBrkr,Lvl,Plywood,Hip,Normal,Norm,1Story,BrkFace,GasA,...,NAmes,NaN,RL,NaN,CBlock,Corner,CompShg,Pave,1Fam,Stone
1,Attchd,SBrkr,Lvl,VinylSd,Gable,Normal,Feedr,1Story,VinylSd,GasA,...,NAmes,NaN,RH,NaN,CBlock,Inside,CompShg,Pave,1Fam,None
2,Attchd,SBrkr,Lvl,Wd Sdng,Hip,Normal,Norm,1Story,Wd Sdng,GasA,...,NAmes,NaN,RL,Gar2,CBlock,Corner,CompShg,Pave,1Fam,BrkFace
3,Attchd,SBrkr,Lvl,BrkFace,Hip,Normal,Norm,1Story,BrkFace,GasA,...,NAmes,NaN,RL,NaN,CBlock,Corner,CompShg,Pave,1Fam,None
4,Attchd,SBrkr,Lvl,VinylSd,Gable,Normal,Norm,2Story,VinylSd,GasA,...,Gilbert,NaN,RL,NaN,PConc,Inside,CompShg,Pave,1Fam,None


In [30]:
datos_ordinales.head()

,calidad_cocinas,funcionalidad,calidad_chimeneas,acabado_garaje,calidad_garaje,condicion_garaje,acceso_garaje_pavimentado,calidad_piscina,calidad_valla,forma_parcela,tipo_instalaciones,pendiente_parcela,calidad_material_exterior,condicion_material_exterior,altura_sotano,condicion_sotano,sotano_exterior,calidad_sotano_habitable1,calidad_sotano_habitable2,calidad_calefaccion
0,TA,Typ,Gd,Fin,TA,TA,P,NaN,NaN,IR1,AllPub,Gtl,TA,TA,TA,Gd,Gd,BLQ,Unf,Fa
1,TA,Typ,NaN,Unf,TA,TA,Y,NaN,MnPrv,Reg,AllPub,Gtl,TA,TA,TA,TA,No,Rec,LwQ,TA
2,Gd,Typ,NaN,Unf,TA,TA,Y,NaN,NaN,IR1,AllPub,Gtl,TA,TA,TA,TA,No,ALQ,Unf,TA
3,Ex,Typ,TA,Fin,TA,TA,Y,NaN,NaN,Reg,AllPub,Gtl,Gd,TA,TA,TA,No,ALQ,Unf,Ex
4,TA,Typ,TA,Fin,TA,TA,Y,NaN,MnPrv,IR1,AllPub,Gtl,TA,TA,Gd,TA,No,GLQ,Unf,Gd


Nos aseguramos que todas las columnas están clasificadas como categóricas o numéricas

In [31]:
[col for col in ames.columns if col not in datos_numericos.columns and\
 col not in datos_categoricos.columns and \
 col not in datos_ordinales.columns]

['precio_venta']

## Variables numéricas

con las variables numéricas muy sencillo, vamos a eliminar valores inexistentes (reemplazandolos pr normalizarlas

In [32]:
from sklearn.preprocessing import Imputer, normalize


# Recordatorio: los transformadores de sklearn devuelven numpy arrays, no dataframes
datos_numericos_imputados_normalizados = pd.DataFrame(
    normalize(Imputer(strategy="median").fit_transform(datos_numericos)),
    columns=datos_numericos.columns
)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


### Variables categóricas

Con las variables categóricas tenemos dos opciones principalmente:
- Usar `sklearn.preprocessing.LabelBinarizer` para hacer un 1 hot encoding y codificarlas como vectores
- Usar la función get_dummies de pandas.

En general la opción recomendada es la de usar `LabelBinarizer` ya que esto nos crea un transformador de scikit learn que podemos usar en pipelines y para transformar nuevas observaciones. Sin embargo dado que esta sección es sobre selección de variables y para no hacerla demasiado larga, directamente voy a usar `pd.get_dummies`.

In [33]:
datos_categoricos_dummy = pd.get_dummies(datos_categoricos, drop_first=True)

In [34]:
datos_categoricos_dummy.shape

(2930, 153)

### Variables ordinales

Yendo al [diccionario de datos](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) (donde se describen todas las variables) hay varias variables que son ordinales (Ordinal), básicamente evaluaciones de una condición de la casa definidas en una escala no numérica pero que tiene cierto orden (de "mejor" a "peor").

Scikit-learn no tiene un Codificador ordinal (hasta ahora hemos usado el label encoder.) Tenemos tres opciones:

1. Implementar nuestro propio `OrdinalEncoder`.
2. Utilizar la implementación de `OrdinalEncoder` del paquete [scikit-learn.contrib.categorical_encoding](https://github.com/scikit-learn-contrib/categorical-encoding), que podemos instalar con 

```conda install -c conda-forge category_encoders
ó
pip install category_encoders
```
3. Ya que estamos usando pandas, podemos convertir las variables ordinales a categorías de pandas, especificar sus niveles y convertirlas a su valor numérico. Vamos a usar esta opción.

In [35]:
dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
}

In [36]:
for columna_ordinal, valores in dict_var_ordinales.items():
    datos_ordinales[columna_ordinal] = (
    datos_ordinales[columna_ordinal]
    .astype("category")
    .cat.set_categories(valores)
    .cat.codes
)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [37]:
datos_ordinales.head()

,calidad_cocinas,funcionalidad,calidad_chimeneas,acabado_garaje,calidad_garaje,condicion_garaje,acceso_garaje_pavimentado,calidad_piscina,calidad_valla,forma_parcela,tipo_instalaciones,pendiente_parcela,calidad_material_exterior,condicion_material_exterior,altura_sotano,condicion_sotano,sotano_exterior,calidad_sotano_habitable1,calidad_sotano_habitable2,calidad_calefaccion
0,2,6,4,3,3,3,1,-1,-1,2,3,2,2,2,3,4,4,4,1,1
1,2,6,-1,1,3,3,2,-1,3,3,3,2,2,2,3,3,1,3,2,2
2,3,6,-1,1,3,3,2,-1,-1,2,3,2,2,2,3,3,1,5,1,2
3,4,6,3,3,3,3,2,-1,-1,3,3,2,3,2,3,3,1,5,1,4
4,2,6,3,3,3,3,2,-1,3,2,3,2,2,2,4,3,1,6,1,3


In [38]:
### Unimos los tres tipos de datos

ames_procesado = pd.concat([
    datos_numericos_imputados_normalizados,
    datos_categoricos_dummy,
    datos_ordinales
], axis=1)

In [39]:
ames_procesado.shape

(2930, 209)

Ya tenemos un dataset preparado para poder entrenar modelos. Este dataset tiene una complejidad dimensional alta ( 209 variables independientes), por lo que vamos a usar técnicas de selección de variables para reducirla.

Antes que nada vamos a ver que errores obtenemos con diversos modelos entrenando con el dataset con todas las variables.

In [40]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


ames_X = ames_procesado
ames_y = ames[variable_objetivo]

Vamos a usar la función `cross_validate` que es una versión más flexible que `cross_val_score`. Evaluaremos usando la raíz del error cuadrático medio (RMSE)

In [41]:
def rmse(y_real, y_pred):
    return np.sqrt(mean_squared_error(y_real, y_pred))

def rmse_cv(estimator, X, y):
    y_pred = estimator.predict(X)
    return rmse(y, y_pred)

res = cross_validate(LinearRegression(), ames_procesado, ames[variable_objetivo],
                     scoring=rmse_cv, n_jobs=-1, cv=10)

Dicha función devuelve no solo las puntuaciones en los datos de testing, sino también en los datos de entrenamiento. Además, nos devuelve el tiempo que tarda dicho estimador en ajustar el modelo y en hacer predicciones con el mismo.

In [42]:
res

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.20307589, 0.20307589, 0.20307589, 0.20307589, 0.12497282,
        0.10934973, 0.10934758, 0.17183137, 0.09372425, 0.09372568]),
 'score_time': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.01562309, 0.01562238, 0.        , 0.        , 0.        ]),
 'test_score': array([30695.34993646, 36428.50813892, 25008.85168604, 30225.68779622,
        33519.50148473, 37583.98991416, 38130.249639  , 36040.64181707,
        31454.11745691, 30946.74565373]),
 'train_score': array([27000.04190286, 26303.34836791, 27653.52874198, 27084.18421673,
        27681.61396358, 26791.51076946, 26600.19518926, 26580.10313152,
        26869.87702619, 27521.10920957])}

In [43]:
def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring=rmse_cv, n_jobs=-1, cv=10, return_train_score=True)
    return resultados_estimador

In [44]:
resultados = {}

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].min()
    return resultados_df

In [45]:
resultados["reg_lineal_sin_seleccion"] = evaluar_modelo(LinearRegression(), ames_X, ames_y)
resultados["svr_sin_seleccion"] = evaluar_modelo(SVR(), ames_X, ames_y)
resultados["rf_sin_seleccion"] = evaluar_modelo(RandomForestRegressor(), ames_X, ames_y)

In [46]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.143713,0.003125,33003.364352,27008.551252,1.000000,1.000168,1.000000,1.929899
svr_sin_seleccion,3.088338,0.307742,81044.162691,82161.012024,21.489545,98.496986,2.455633,5.870825
rf_sin_seleccion,1.004451,0.003124,33045.353662,13994.798768,6.989262,1.000000,1.001272,1.000000


Hay 3 tipos generales de estrategias de selección de variables:

# Métodos de filtrado

Los métodos de filtrado usan métodos estadísticos para seleccionar las variables que proporcionan la mayor cantidad de información. Estos métodos se aplican de forma previa a entrenar el modelo (preprocesado), y **son completamente independientes de la elección del estimador**. Generalmente funcionan definiendo una función de evaluación $S(xk_i, y_k)$, evaluando cada variable independiente para cada observación respecto a la variable objetivo de dicha observación, y eligiendo aquellas `K` variables que mejor funcionan.



Scikit-learn tiene las siguientes funciones de evaluación:

- Para regresión: `f_regression, mutual_info_regression`
- Para clasificación: `chi2, f_classif, mutual_info_classif`

f_regression y f_classif devuelven [estadísticos F](https://es.wikipedia.org/wiki/Estad%C3%ADstico_F) (F-values), entrenando un modelo lineal entre las variables independientes y la objetivo en el caso de regresión, y un test ANOVA en el caso de clasificación.

`mutual_info_regression` y `mutual_info_classif` computan el coeficiente de información mutua (MIC) entre las variables independientes y la variable objetivo.

El coeficiente de información mutua nos da una medida de la dependencia entre las variables. El MIC entre dos variables es 0 si no hay relación entre las mismas, y aumenta conforme más relación tienen.

El MIC se define como:

$$I(X;Y) = \sum_{x,y} P_{XY}(x,y) \log
{P_{XY}(x,y) \over P_X(x) P_Y(y)}$$

con $P_{XY}$ siendo la probabilidad conjunta de X e Y


El evaluador `chi2` calcula el estadístico chi cuadrado y lo convierte a estadísticos F, con el estadístico chi cuadrado definido como:
  $$\chi^2(x,y) = \sum {\frac{(x-y)^2}{y}}$$


In [47]:
from sklearn.feature_selection import SelectKBest, f_regression

In [48]:
selector_kbest10 = SelectKBest(f_regression, k=10)
ames_X_kbest10 = selector_kbest10.fit_transform(ames_X, ames_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
ames_X_kbest10.shape

(2930, 10)

La funcion `get_support` nos devuelve un vector booleano (True/False), aquellos elementos con True son las columnas que se han seleccionado

In [50]:
columnas_seleccion_kbest10 = ames_X.loc[:,selector_kbest10.get_support()].columns
columnas_seleccion_kbest10

Index(['barrio_NridgHt', 'tipo_andamios_PConc', 'tipo_revestimiento_None',
       'calidad_cocinas', 'calidad_chimeneas', 'acabado_garaje',
       'calidad_material_exterior', 'altura_sotano', 'sotano_exterior',
       'calidad_calefaccion'],
      dtype='object')

El parámetro `scores_` del selector nos devuelve los resultados de la función de evaluación

In [51]:
selector_kbest10.scores_[:10]

array([ 98.54044172,  46.4426689 , 140.45170476,  11.38580355,
       507.38479194, 470.33287137, 495.34300746, 320.65956248,
       215.58416354,   3.89847629])

In [52]:
puntuaciones_seleccion_kbest10 = zip(ames_X.columns, selector_kbest10.scores_,
                                     selector_kbest10.get_support())
evaluacion_kbest10 = sorted(
                filter(lambda c: c[2], puntuaciones_seleccion_kbest10),
                key=lambda c: c[1], reverse=True
)

In [53]:
list(evaluacion_kbest10)

[('calidad_material_exterior', 2781.405629339341, True),
 ('calidad_cocinas', 2423.0002986125946, True),
 ('altura_sotano', 1372.4642743061154, True),
 ('acabado_garaje', 1163.3451168655752, True),
 ('calidad_chimeneas', 1124.6033740020175, True),
 ('tipo_andamios_PConc', 1090.6972231859302, True),
 ('calidad_calefaccion', 744.819490297256, True),
 ('barrio_NridgHt', 676.7927453491437, True),
 ('sotano_exterior', 604.3556456392505, True),
 ('tipo_revestimiento_None', 570.9821876809164, True)]

Esto nos permite ver cual es la puntuación que le da el evaluador `f_regression` a cada variable independiente

In [54]:
resultados["reg_lineal_kbest_10"] = evaluar_modelo(LinearRegression(), ames_X_kbest10, ames_y)
resultados["rf_kbest_10"] = evaluar_modelo(RandomForestRegressor(), ames_X_kbest10, ames_y)
resultados["svr_kbest_10"] = evaluar_modelo(SVR(), ames_X_kbest10, ames_y)

In [55]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.143713,0.003125,33003.364352,27008.551252,inf,2.000733,1.000000,1.929899
svr_sin_seleccion,3.088338,0.307742,81044.162691,82161.012024,inf,197.033064,2.455633,5.870825
rf_sin_seleccion,1.004451,0.003124,33045.353662,13994.798768,inf,2.000397,1.001272,1.000000
reg_lineal_kbest_10,0.000000,0.001562,44744.490966,44962.297898,NaN,1.000000,1.355755,3.212786
rf_kbest_10,0.051552,0.001562,46979.833761,31704.177502,inf,1.000137,1.423486,2.265426
svr_kbest_10,0.440524,0.028119,80908.503317,82043.551184,inf,18.003312,2.451523,5.862432


Vemos que al reducir las dimensiones a solo 10, los modelos lineales y RF funcionan bastante peor, aunque vemos que por otra parte las diferencias entre la evaluación de entrenamiento y test se han reducido en todos los casos. Esto es así por que al tener menos variables, hay mucha menos flexibilidad para sobreajustar (es decir, "memorizar" los datos de entrenamiento). SVM sin embargo funciona más o menos igual (de mal), pero los tiempos de entrenamiento se reducen considerablemente (de 3.9 segundos a 0.7).

Probamos ahora para 50 variables

In [62]:
selector_kbest50 = SelectKBest(f_regression, k=50)
ames_X_kbest50 = selector_kbest50.fit_transform(ames_X, ames_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [63]:
resultados["reg_lineal_kbest_50"] = evaluar_modelo(LinearRegression(), ames_X_kbest50, ames_y)
resultados["rf_kbest_50"] = evaluar_modelo(RandomForestRegressor(), ames_X_kbest50, ames_y)
resultados["svr_kbest_50"] = evaluar_modelo(SVR(), ames_X_kbest50, ames_y)

In [64]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.143713,0.003125,33003.364352,27008.551252,inf,inf,1.000000,1.929899
svr_sin_seleccion,3.088338,0.307742,81044.162691,82161.012024,inf,inf,2.455633,5.870825
rf_sin_seleccion,1.004451,0.003124,33045.353662,13994.798768,inf,inf,1.001272,1.000000
reg_lineal_kbest_10,0.000000,0.001562,44744.490966,44962.297898,NaN,inf,1.355755,3.212786
rf_kbest_10,0.051552,0.001562,46979.833761,31704.177502,inf,inf,1.423486,2.265426
svr_kbest_10,0.440524,0.028119,80908.503317,82043.551184,inf,inf,2.451523,5.862432
reg_lineal_kbest_50,0.024414,0.000000,34738.437910,33794.489434,inf,NaN,1.052573,2.414789
rf_kbest_50,0.474887,0.003126,34950.166603,14577.247648,inf,inf,1.058988,1.041619
svr_kbest_50,0.923220,0.071862,80971.039786,82091.449480,inf,inf,2.453418,5.865854


Vemos que para 50 variables, los tres estimadores funcionan de forma similar al modelo entrenado con las 209 variables iniciales, sin embargo los tiempos de entrenamiento de los mismos se reducen considerablemente.

De forma similar a `SelectKBest`, donde le decimos al transformador el número de variables que queremos mantener, podemos usar [`SelectPercentile`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile) donde especificamos el porcentaje de variables sobre el total.

Por ejemplo, si solo quisieramos conservar el 10% de las variables (sin importar que tengamos 200 o 20000), podemos hacer lo siguiente:


In [65]:
from sklearn.feature_selection import SelectPercentile

selector_pct10 = SelectPercentile(f_regression, percentile=10)
ames_X_pct10 = selector_pct10.fit_transform(ames_X, ames_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [66]:
ames_X_pct10.shape

(2930, 21)

El transformador nos selecciona 21 variables (el 10 por ciento de 209 redondeado)

# Métodos envolventes (wrapper methods)

Los métodos envolventes *(wrapper methods)* funcionan de forma similar a los métodos de ranking. Sin embargo, en lugar de usar una función estadística independiente del modelo para evaluar las variables, estos métodos usan la función de evaluación o el performance de los modelos como input para decidir que variables elegir (es decir, "envuelven" el funcionamiento del estimador). Ésto significa que los métodos de filtrado se pueden aplicar independientemente de la elección del modelo, ya que consideran los modelos como una caja negra que produce evaluaciones, aunque claro, diferentes modelos producirán diferentes selecciones de variables.

`Scikit-learn` implementa un metodo envolvente llamado **Recursive Feature Elimination [(RFE)](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)**, o Eliminación Recursiva de Variables. RFE funciona seleccionando todas las variables, entrenando el modelo, usando los coeficientes `coef_` o la importancia de las variables `feature_importances_` en función del estimador, y eliminando n variables. Este proceso se repite hasta que se alcanza el número de variables deseado.

In [67]:
from sklearn.feature_selection import RFE
estimador_selector = RandomForestRegressor()
selector_rfe10_rf = RFE(estimador_selector, n_features_to_select=10)
ames_X_rfe10_rf = selector_rfe10_rf.fit_transform(ames_X, ames_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

In [68]:
ames_X_rfe10_rf.shape

(2930, 10)

In [69]:
evaluacion_rfe10_rf = sorted(
    filter(lambda c: c[2], 
        zip(
            ames_X.columns,
            selector_rfe10_rf.ranking_,
            selector_rfe10_rf.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)

In [70]:
evaluacion_rfe10_rf

[('condicion_general', 1, True),
 ('ano_remodelado', 1, True),
 ('area_sotano_habitable1', 1, True),
 ('area_habitable_sobre_suelo', 1, True),
 ('area_garage', 1, True),
 ('ano_venta', 1, True),
 ('calidad_cocinas', 1, True),
 ('calidad_chimeneas', 1, True),
 ('calidad_material_exterior', 1, True),
 ('altura_sotano', 1, True)]

In [71]:
resultados["reg_lineal_rfe10_rf"] = evaluar_modelo(LinearRegression(), ames_X_rfe10_rf, ames_y)
resultados["rf_rfe10_rf"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfe10_rf, ames_y)
resultados["svr_rfe10_rf"] = evaluar_modelo(SVR(), ames_X_rfe10_rf, ames_y)

Si usamos otro estimador para evaluar veremos que las variables elegidas pueden ser completamente distintas. Los estimadores que se pueden usar tienen que implementar el metodo `coef_` o el metodo `feature_importance` (es decir, tienen que tener una manera de ordenar variables en función de su importancia). Por ejemplo, no podemos usar SVMs.

In [72]:
estimador_selector = LinearRegression()
selector_rfe10_lineal = RFE(estimador_selector, n_features_to_select=10)
ames_X_rfe10_lineal = selector_rfe10_lineal.fit_transform(ames_X, ames_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [73]:
evaluacion_rfe10_lineal = sorted(
    filter(lambda c: c[2], 
        zip(
            ames_X.columns,
            selector_rfe10_lineal.ranking_,
            selector_rfe10_lineal.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)
evaluacion_rfe10_lineal

[('calidad_general', 1, True),
 ('condicion_general', 1, True),
 ('n_medios_aseos_sotanos', 1, True),
 ('n_aseos_sobre_suelo', 1, True),
 ('n_dormitorios_sobre_suelo', 1, True),
 ('n_cocinas', 1, True),
 ('n_habitaciones_sobre_suelo', 1, True),
 ('n_chimeneas', 1, True),
 ('n_coches_garaje', 1, True),
 ('ano_venta', 1, True)]

In [74]:
set(evaluacion_rfe10_rf).intersection(set(evaluacion_rfe10_lineal))

{('ano_venta', 1, True), ('condicion_general', 1, True)}

In [75]:
resultados["reg_lineal_rfe10_lineal"] = evaluar_modelo(LinearRegression(), ames_X_rfe10_lineal, ames_y)
resultados["rf_rfe10_lineal"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfe10_lineal, ames_y)
resultados["svr_rfe10_lineal"] = evaluar_modelo(SVR(), ames_X_rfe10_lineal, ames_y)

In [76]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.143713,0.003125,33003.364352,27008.551252,inf,inf,1.000000,1.957683
svr_sin_seleccion,3.088338,0.307742,81044.162691,82161.012024,inf,inf,2.455633,5.955343
rf_sin_seleccion,1.004451,0.003124,33045.353662,13994.798768,inf,inf,1.001272,1.014396
reg_lineal_kbest_10,0.000000,0.001562,44744.490966,44962.297898,NaN,inf,1.355755,3.259039
rf_kbest_10,0.051552,0.001562,46979.833761,31704.177502,inf,inf,1.423486,2.298040
svr_kbest_10,0.440524,0.028119,80908.503317,82043.551184,inf,inf,2.451523,5.946829
reg_lineal_kbest_50,0.024414,0.000000,34738.437910,33794.489434,inf,NaN,1.052573,2.449553
rf_kbest_50,0.474887,0.003126,34950.166603,14577.247648,inf,inf,1.058988,1.056614
svr_kbest_50,0.923220,0.071862,80971.039786,82091.449480,inf,inf,2.453418,5.950301
reg_lineal_rfe10_rf,0.004684,0.003124,37839.400772,37812.707502,inf,inf,1.146532,2.740809


Vemos que el estimador RF entrenado con solo 10 variables seleccionadas por `RFE` basado en RF es el modelo que mejor funciona, y tarda 8 veces menos en entrenar.

scikit-learn también tiene la implementación [RFECV](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV) que implementa el mismo algoritmo pero hace validación cruzada y entrena en múltiples particiones del dataset para calcular automáticamente el conjunto de variables que funciona mejor (a costa de un mayor tiempo de procesamiento).

In [77]:
from sklearn.feature_selection import RFECV

estimador_selector = RandomForestRegressor()
selector_rfecv = RFECV(estimador_selector)
ames_X_rfecv = selector_rfecv.fit_transform(ames_X, ames_y)

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of 

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

...

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\jorge\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in versio

In [78]:
ames_X_rfecv.shape

(2930, 188)

En este caso `RFECV` ha elegido 188 variables como aquellas que proporcionan el mejor funcionamiento al estimador RF utilizado.

In [79]:
resultados["reg_lineal_rfecv_rf"] = evaluar_modelo(LinearRegression(), ames_X_rfecv, ames_y)
resultados["rf_rfecv_rf"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfecv, ames_y)
resultados["svr_rfecv_rf"] = evaluar_modelo(SVR(), ames_X_rfecv, ames_y)

In [80]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.143713,0.003125,3.300336e+04,27008.551252,inf,inf,1.000000e+00,1.957683
svr_sin_seleccion,3.088338,0.307742,8.104416e+04,82161.012024,inf,inf,2.455633e+00,5.955343
rf_sin_seleccion,1.004451,0.003124,3.304535e+04,13994.798768,inf,inf,1.001272e+00,1.014396
reg_lineal_kbest_10,0.000000,0.001562,4.474449e+04,44962.297898,NaN,inf,1.355755e+00,3.259039
rf_kbest_10,0.051552,0.001562,4.697983e+04,31704.177502,inf,inf,1.423486e+00,2.298040
svr_kbest_10,0.440524,0.028119,8.090850e+04,82043.551184,inf,inf,2.451523e+00,5.946829
reg_lineal_kbest_50,0.024414,0.000000,3.473844e+04,33794.489434,inf,NaN,1.052573e+00,2.449553
rf_kbest_50,0.474887,0.003126,3.495017e+04,14577.247648,inf,inf,1.058988e+00,1.056614
svr_kbest_50,0.923220,0.071862,8.097104e+04,82091.449480,inf,inf,2.453418e+00,5.950301
reg_lineal_rfe10_rf,0.004684,0.003124,3.783940e+04,37812.707502,inf,inf,1.146532e+00,2.740809


Vemos que el método que funciona mejor es el de RFECV usando RF como estimador. Este método funciona mejor que  el segundo mejor modelo, un Random Forest entrenado con todo el dataset, pero se entrena en menos tiempo (0.81 segundos versus  0.99 segundos)

# Metodos embebidos (Embedded Methods)

Éstos métodos son metodos de selección de variables que están integrados dentro del proceso de entrenamiento de modelos, y son específicos para cada modelo.

El ejemplo clásico de método embebidos de selección de variables son los procesos de regularización en regresión lineal. Por ejemplo el método L1 tiende a convertir los coeficientes de las variables que no funcionan a 0, por lo tanto eliminandolas.